In [1]:
from datetime import timedelta, date
import pandas as pd
import numpy as np
import quandl, math
from dateutil.relativedelta import relativedelta
from utils import expiration_date, is_business_day, is_holiday

quandl.ApiConfig.api_key = "tjxFewEnckmn8zjjsHSZ"

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
start_date = date(2015, 1, 1)
end_date = date(2015, 1, 10)
signal = []
spot_price = []
future_price = []


spot = quandl.get("EIA/PET_RWTC_D",start_date=start_date,end_date=end_date,returns="numpy")

future = quandl.get("CHRIS/CME_CL1",start_date=start_date,end_date=end_date,returns="numpy")

for item in spot:
    s_price = float(item[1])
    spot_price.extend([s_price])
    
for item in future: 
    f_price = float(item[4])
    future_price.extend([f_price])

    
for single_date in daterange(start_date, end_date):
    sig = trade_signal(single_date,"WTI",6)
    signal.extend([sig])
    

ModuleNotFoundError: No module named 'utils'

In [ ]:
retrn = np.zeros(len(spot_price),dtype=np.float)
retrn[0] = 0
pnl = np.zeros(len(spot_price),dtype=np.float)
pnl[0] = 0
Init_capital = 1000
capital = np.zeros(len(signal),dtype=np.float)
capital[0]=Init_capital


for i in range(1,len(spot_price)-1):
    if abs(signal[i])>.01 and abs(signal[i])<2:
        retrn[i] = np.sign(signal[i])*(((spot_price[i]-future_price[i])/future_price[i])+((spot_price[i]-spot_price[i-1])/spot_price[i-1]))
        capital[i]=(1+retrn[i])*capital[i-1]
        pnl[i]=capital[i]-capital[i-1]
        total_PNL=np.cumsum(pnl)
    else:
        retrn[i]=0
        pnl[i]=pnl[i-1]
        
        